In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
sys.path.append("../../../..") # relative path to module toolkit

In [3]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from toolkit import getLabelsFromDir, plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight 
from keras.backend import clear_session
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
# setup
batch_size = 3
"""
Genus, hvor der kun er en species, så vi behøver ikke at træne modellen.

"Achenium",
"Cafius",
"Creophilus",
"Dinothenarus",
"Emus",
"Erichsonius",
"Euryporus",
"Gabronthus",
"Gauropterus",
"Lobrathium",
"Megalinus",
"Nudobius",
"Paederidus",
"Phacophallus",
"Remus",
"Velleius"
"""


for genus in [
    "Lithocharis",
    "Atrecus",
    "Pseudomedon",
    "Sunius",
    "Astenus",
    "Heterothops",
    "Ocypus",
    "Gabrius",
    "Staphylinus",
    "Acylophorus",
    "Quedius",
    "Philonthus",
    "Medon",
    "Neobisnius",
    "Lathrobium",
    "Paederus",
    "Leptacinus",
    "Platydracus",
    "Ochthephilum",
    "Othius",
    "Gyrohypnus",
    "Bisnius",
    "Tetartopeus",
    "Xantholinus",
    "Rugilus",
    "Tasgius",
    "Ontholestes",
    "Scopaeus"]:

    train_dir = "../../../../../images/images_genus_species_shuffled/train/" + genus + "/"
    val_dir = "../../../../../images/images_genus_species_shuffled/val/" + genus + "/"
    
    labels = getLabelsFromDir(train_dir)

    train_datagen = ImageDataGenerator(rescale=1./255.)
    val_datagen = ImageDataGenerator(rescale=1./255.)
    
    
    train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(416, 416),
                                                    shuffle=True,
                                                    seed=1)
    val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(416, 416),
                                                    shuffle=True,
                                                    seed=1)

    number_of_classes = train_generator.num_classes
    train_images = train_generator.samples
    val_images = val_generator.samples
    save_plot_name = genus + "_trainplot.png"
    model_name = genus + '_resNet50pretrained.h5'
    
    steps_per_epoch = int(train_images/batch_size) + 1
    validation_steps = int(val_images/batch_size) + 1
    
    
    ### Model building ####
    base_model = ResNet50(
                include_top=False,
                input_shape=(416, 416, 3),
                weights="imagenet")

    #add a new dense layer to the end of the network inplace of the old layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    # add the outplut layer
    predictions = Dense(number_of_classes, activation='softmax')(x)

    # create new model composed of pre-trained network and new final layers
    model = Model(input=base_model.input, output=predictions)

    for layer in model.layers:
        layer.trainable = True
    
    # compile model
    model.compile(loss='categorical_crossentropy',
            optimizer='sgd',
            metrics=['accuracy'])
    
    filepath = model_name

    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only=True)

    early_stop = EarlyStopping(monitor='val_acc', mode='max', patience=10)

    callbacks = [checkpoint, early_stop]
    
    
    the_class_weight = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)

    #Train
    history = model.fit_generator(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    class_weight=the_class_weight)
    
    """ 
    plt.plot(history.history['acc'], label='training accuracy')
    plt.plot(history.history['val_acc'], label='validation accuracy')
    plt.title('Accuracy')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig(save_plot_name, bbox_inches='tight')
    """
    clear_session()
    del model

Found 198 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


/home/herri/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/home/herri/.local/lib/python3.5/site-packages/ipykernel_launcher.py:105: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Epoch 1/100
67/67 [==============================] - 21s 309ms/step - loss: 0.4127 - acc: 0.8109 - val_loss: 4.0769 - val_acc: 0.6842

Epoch 00001: val_acc improved from -inf to 0.68421, saving model to Lithocharis_resNet50pretrained.h5
Epoch 2/100
67/67 [==============================] - 15s 219ms/step - loss: 0.2197 - acc: 0.9154 - val_loss: 3.1948 - val_acc: 0.7368

Epoch 00002: val_acc improved from 0.68421 to 0.73684, saving model to Lithocharis_resNet50pretrained.h5
Epoch 3/100
67/67 [==============================] - 15s 219ms/step - loss: 0.1475 - acc: 0.9403 - val_loss: 0.5002 - val_acc: 0.7719

Epoch 00003: val_acc improved from 0.73684 to 0.77193, saving model to Lithocharis_resNet50pretrained.h5
Epoch 4/100
67/67 [==============================] - 15s 219ms/step - loss: 0.0747 - acc: 0.9652 - val_loss: 4.8089 - val_acc: 0.7018

Epoch 00004: val_acc did not improve from 0.77193
Epoch 5/100
67/67 [==============================] - 15s 220ms/step - loss: 0.0797 - acc: 0.9801 -


Epoch 00012: val_acc did not improve from 1.00000
Epoch 13/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0078 - acc: 1.0000 - val_loss: 0.0722 - val_acc: 0.9756

Epoch 00013: val_acc did not improve from 1.00000
Epoch 14/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0077 - acc: 1.0000 - val_loss: 0.2503 - val_acc: 0.9268

Epoch 00014: val_acc did not improve from 1.00000
Epoch 15/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0359 - acc: 0.9867 - val_loss: 0.4318 - val_acc: 0.7805

Epoch 00015: val_acc did not improve from 1.00000
Epoch 16/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0140 - acc: 1.0000 - val_loss: 0.2591 - val_acc: 0.8780

Epoch 00016: val_acc did not improve from 1.00000
Epoch 17/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0178 - acc: 0.9933 - val_loss: 0.3116 - val_acc: 0.8293

Epoch 00017: val_acc did not improve from 1.00000
Epoch 18/100
50/50 

139/139 [==============================] - 30s 219ms/step - loss: 0.0113 - acc: 1.0000 - val_loss: 0.5620 - val_acc: 0.8584

Epoch 00012: val_acc did not improve from 0.85841
Epoch 13/100
139/139 [==============================] - 30s 219ms/step - loss: 0.0114 - acc: 0.9976 - val_loss: 0.5861 - val_acc: 0.8407

Epoch 00013: val_acc did not improve from 0.85841
Epoch 14/100
139/139 [==============================] - 30s 219ms/step - loss: 0.0277 - acc: 0.9928 - val_loss: 0.5341 - val_acc: 0.7876

Epoch 00014: val_acc did not improve from 0.85841
Epoch 15/100
139/139 [==============================] - 30s 219ms/step - loss: 0.0140 - acc: 1.0000 - val_loss: 0.7522 - val_acc: 0.8142

Epoch 00015: val_acc did not improve from 0.85841
Epoch 16/100
139/139 [==============================] - 30s 219ms/step - loss: 0.0094 - acc: 0.9976 - val_loss: 0.6672 - val_acc: 0.8496

Epoch 00016: val_acc did not improve from 0.85841
Epoch 17/100
139/139 [==============================] - 30s 219ms/step - 

...

50/50 [==============================] - 11s 217ms/step - loss: 0.0039 - acc: 1.0000 - val_loss: 0.3706 - val_acc: 0.9000

Epoch 00020: val_acc did not improve from 0.92500
Epoch 21/100
50/50 [==============================] - 11s 217ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.3808 - val_acc: 0.9000

Epoch 00021: val_acc did not improve from 0.92500
Epoch 22/100
50/50 [==============================] - 11s 218ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.3689 - val_acc: 0.9250

Epoch 00022: val_acc improved from 0.92500 to 0.92500, saving model to Othius_resNet50pretrained.h5
Epoch 23/100
50/50 [==============================] - 11s 217ms/step - loss: 0.0029 - acc: 1.0000 - val_loss: 0.3610 - val_acc: 0.9250

Epoch 00023: val_acc did not improve from 0.92500
Epoch 24/100
50/50 [==============================] - 11s 217ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.3674 - val_acc: 0.9000

Epoch 00024: val_acc did not improve from 0.92500
Epoch 25/100
50/50 [=============

245/245 [==============================] - 54s 219ms/step - loss: 0.1059 - acc: 0.9701 - val_loss: 0.2894 - val_acc: 0.9212

Epoch 00006: val_acc improved from 0.75369 to 0.92118, saving model to Bisnius_resNet50pretrained.h5
Epoch 7/100
245/245 [==============================] - 54s 219ms/step - loss: 0.1501 - acc: 0.9456 - val_loss: 2.5069 - val_acc: 0.5074

Epoch 00007: val_acc did not improve from 0.92118
Epoch 8/100
245/245 [==============================] - 54s 219ms/step - loss: 0.1146 - acc: 0.9619 - val_loss: 3.2207 - val_acc: 0.3300

Epoch 00008: val_acc did not improve from 0.92118
Epoch 9/100
245/245 [==============================] - 54s 219ms/step - loss: 0.0464 - acc: 0.9878 - val_loss: 0.8324 - val_acc: 0.7635

Epoch 00009: val_acc did not improve from 0.92118
Epoch 10/100
245/245 [==============================] - 54s 219ms/step - loss: 0.0206 - acc: 0.9986 - val_loss: 0.1842 - val_acc: 0.9458

Epoch 00010: val_acc improved from 0.92118 to 0.94581, saving model to Bisn

95/95 [==============================] - 21s 219ms/step - loss: 5.3818e-04 - acc: 1.0000 - val_loss: 0.3679 - val_acc: 0.9383

Epoch 00026: val_acc did not improve from 0.98765
Epoch 27/100
95/95 [==============================] - 21s 219ms/step - loss: 3.6833e-04 - acc: 1.0000 - val_loss: 0.2663 - val_acc: 0.9630

Epoch 00027: val_acc did not improve from 0.98765
Found 196 images belonging to 6 classes.
Found 51 images belonging to 6 classes.
Epoch 1/100
66/66 [==============================] - 19s 282ms/step - loss: 1.6044 - acc: 0.4041 - val_loss: 1.2902 - val_acc: 0.4444

Epoch 00001: val_acc improved from -inf to 0.44444, saving model to Xantholinus_resNet50pretrained.h5
Epoch 2/100
66/66 [==============================] - 14s 217ms/step - loss: 0.9801 - acc: 0.6365 - val_loss: 1.5668 - val_acc: 0.4259

Epoch 00002: val_acc did not improve from 0.44444
Epoch 3/100
66/66 [==============================] - 14s 217ms/step - loss: 0.5133 - acc: 0.8283 - val_loss: 0.8571 - val_acc: 0.6

164/164 [==============================] - 36s 220ms/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.0480 - val_acc: 0.9787

Epoch 00019: val_acc did not improve from 0.99291
Epoch 20/100
164/164 [==============================] - 36s 220ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0416 - val_acc: 0.9716

Epoch 00020: val_acc did not improve from 0.99291
Found 120 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Epoch 1/100
41/41 [==============================] - 13s 319ms/step - loss: 1.1009 - acc: 0.5122 - val_loss: 1.5358 - val_acc: 0.4375

Epoch 00001: val_acc improved from -inf to 0.43750, saving model to Tasgius_resNet50pretrained.h5
Epoch 2/100
41/41 [==============================] - 9s 217ms/step - loss: 0.5212 - acc: 0.8130 - val_loss: 0.9232 - val_acc: 0.5313

Epoch 00002: val_acc improved from 0.43750 to 0.53125, saving model to Tasgius_resNet50pretrained.h5
Epoch 3/100
41/41 [==============================] - 9s 218ms/step - loss: 0.2437 - acc:

48/48 [==============================] - 10s 217ms/step - loss: 0.0653 - acc: 0.9656 - val_loss: 4.0729 - val_acc: 0.5000

Epoch 00009: val_acc did not improve from 0.92105
Epoch 10/100
48/48 [==============================] - 10s 217ms/step - loss: 0.0833 - acc: 0.9722 - val_loss: 0.5232 - val_acc: 0.7368

Epoch 00010: val_acc did not improve from 0.92105
Epoch 11/100
48/48 [==============================] - 10s 217ms/step - loss: 0.0787 - acc: 0.9861 - val_loss: 0.4338 - val_acc: 0.7895

Epoch 00011: val_acc did not improve from 0.92105
Epoch 12/100
48/48 [==============================] - 10s 217ms/step - loss: 0.1891 - acc: 0.9169 - val_loss: 1.9153 - val_acc: 0.1579

Epoch 00012: val_acc did not improve from 0.92105
Epoch 13/100
48/48 [==============================] - 10s 217ms/step - loss: 0.0407 - acc: 1.0000 - val_loss: 0.1843 - val_acc: 0.9474

Epoch 00013: val_acc improved from 0.92105 to 0.94737, saving model to Scopaeus_resNet50pretrained.h5
Epoch 14/100
48/48 [===========